<a href="https://colab.research.google.com/github/MammadTavakoli/CDR-Pumping-Fraud/blob/master/Youtubedl_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Pakages

In [1]:
# !pip install pytube
! pip install git+https://github.com/pytube/pytube
!pip install youtube-transcript-api

  Cloning https://github.com/pytube/pytube to /tmp/pip-req-build-q4f6jilz
  Running command git clone --filter=blob:none --quiet https://github.com/pytube/pytube /tmp/pip-req-build-q4f6jilz
  Resolved https://github.com/pytube/pytube to commit a32fff39058a6f7e5e59ecd06a7467b71197ce35
  Preparing metadata (setup.py) ... done
  Created wheel for pytube: filename=pytube-15.0.0-py3-none-any.whl size=57579 sha256=6fc8a51a098338a1054d835b43550048d0913fd3bd3ca466eda3115583d8c884
  Stored in directory: /tmp/pip-ephem-wheel-cache-nrrqut8q/wheels/b0/a9/7d/d3579227a695fdd15288c35657b3332ef0d71430ca7f685769
Successfully built pytube


# Import Pakages

In [2]:
from __future__ import unicode_literals

In [3]:
import os
from google.colab import drive

In [4]:
from pytube import YouTube, Playlist
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from youtube_transcript_api.formatters import SRTFormatter

In [5]:
import re
import math
import time
import string
from typing import Dict, Optional

# DOWNLOAD VIDEO

In [6]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [7]:
x = "۰۱۲۳۴۵۶۷۸۹"
y = "0123456789"
table_digit = str.maketrans(x, y)
table_punctuation = str.maketrans(string.punctuation, '-'*len(string.punctuation))

In [8]:
def url_list_add(url):
    # Define the path where the file is located
    path = "//content//drive//MyDrive//YouTube//"

    # Create the complete file path by joining the path and file name
    file_path = os.path.join(path, "_urls.txt")

    # Check if the file exists
    if not os.path.isfile(file_path):
        # If the file doesn't exist, create an empty file
        open(file_path, "w").close()

    # Open the file in read mode
    with open(file_path, 'r', encoding="utf-8") as file:
        # Read the contents of the file
        content = file.read()

        # Check if the url already exists in the file
        if url in content:
            print(f'{url} already exists')
            return False

    # Open the file in append mode
    with open(file_path, "a", encoding="utf-8") as file:
        # Write the url to the file
        file.write(url + "\n")
        print(f'{url} added')

    return True

In [9]:
def create_folder(url, path):
    try:
        # Create the folder
        os.makedirs(path, exist_ok=True)

        # Change the current working directory to the new folder
        os.chdir(path)
        print(os.getcwd())

        # Save the URL in a text file
        url_file_path = os.path.join(path, "_url.txt")
        with open(url_file_path, "w", encoding="utf-8") as f:
            f.write(url)

        print("Folder created successfully.")
    except Exception as e:
        print("Error creating folder:", str(e))

In [10]:
def download_srt(youtube, filename, target_language='en'):
    try:
        # Get video id
        video_id = youtube.video_id

        # Get transcript of video
        srt = YouTubeTranscriptApi.get_transcript(video_id, languages=[target_language])

        # Convert to SRT format
        formatter = SRTFormatter()
        en_caption_data = formatter.format_transcript(srt)

        # Save the subtitle in the file
        print('Downloading subtitles for:', filename)
        with open(f"{filename}.srt", "w", encoding="utf-8") as f:
            f.write(en_caption_data)

        # Access the description attribute of the YouTube object
        description = youtube.description
        with open(f"{filename}.description", "w", encoding="utf-8") as f:
            f.write(description)

    except TranscriptsDisabled:
        print('Subtitles are disabled for:', filename)
    except Exception as e:
        print('Error downloading subtitles for:', filename, 'Error:', str(e))

In [11]:
# Function to download a video from YouTube using pytube
def download_video_by_pytube(url, base_path="//content//drive//MyDrive//YouTube//Download_video//",
                             playlist_start=1, skip_download=False, max_resolution=1280):

    # Check if the URL is for a playlist
    if 'playlist?list=' in url.lower():
        playlist = Playlist(url)
        folder_name = f"{playlist.title}_{playlist.owner}"
        playlist = list(playlist.video_urls)
    else:
        playlist = []
        playlist.append(url)
        youtube = YouTube(url)
        folder_name = f"{youtube.title}_{youtube.author}"

    # Create a clean folder name and path
    folder_name = re.sub(r'[^\w\s-]', '', folder_name).strip()
    folder_name = re.sub(r'[-\s]+', '-', folder_name)
    path = os.path.join(base_path, folder_name)

    # Create the folder for the downloaded videos
    create_folder(url, path)

    for i, video_url in enumerate(playlist[playlist_start-1:]):
        youtube = YouTube(video_url)

        # Get the name of the video
        title = youtube.title.translate(table_punctuation).translate(table_digit)
        j = str(i+playlist_start).zfill(3)
        filename = f"{j}_{title}"
        print(j, ' ', title)


        # print(youtube.streaming_data)

        try:

          video_dict = {}
          for streaming_data in youtube.streaming_data['formats']:
            if 'video/mp4;' in streaming_data['mimeType']:
              video_dict[streaming_data['url']] = int(streaming_data['qualityLabel'].split('p')[0])
              print(streaming_data['quality'], streaming_data['qualityLabel'])

          video_dict = filtered_dict = dict(filter(lambda x: x[1] <= max_resolution, video_dict.items()))
          videos = sorted(video_dict.items(), key=lambda x: x[1], reverse=True)
          print(videos[0][1])


            # Get the streams that have a resolution
          streams = [stream for stream in youtube.streams.filter(progressive=True, file_extension='mp4') if stream.resolution]

          # Sort the streams by resolution in descending order
          streams.sort(key=lambda stream: int(stream.resolution.replace('p', '')), reverse=True)

          # Find the first stream that has a resolution less than or equal to max_resolution
          stream = next((stream for stream in streams if int(stream.resolution.replace('p', '')) <= max_resolution), None)

          # Download the video and subtitle and specify the filename
          if stream:
              if not skip_download:
                  stream.download(filename=f"{filename}.mp4")
                  print("Downloaded resolution is:", stream.resolution)
              download_srt(youtube, filename)

        except Exception as e:
          print(e)

In [12]:
# The URLS list holds the YouTube video or playlist URLs that you want to download.
# Each URL in the list can be either a string (representing a single video URL) or a dictionary (representing a playlist).
# If it's a dictionary, the dictionary key is the URL of the playlist, and the dictionary value is the index of the video in the playlist where downloading should start.
# If the playlist URL is specified as a string, downloading will start from the first video in the playlist.
# This script allows you to download videos from a YouTube playlist to your Google Drive and then to your PC, with the ability to resume the download from the last video if needed.

# Example of the URLS list:
# URLS = [
#     {"https://www.youtube.com/playlist?list=1234": 3},  # Download from the 3rd video of the playlist
#     {"https://www.youtube.com/playlist?list=1234": 1},  # Download from the first video of the playlist (all videos)
#     "https://www.youtube.com/playlist?list=1234",      # Download from the first video of the playlist (all videos)
#     "https://www.youtube.com/watch?v=1234"              # Download a single video
# ]

URLS = [
"https://www.youtube.com/playlist?list=PLD9Qr3ozd9P0cIvxMOybbeyrVr6ki2cvB",

{"https://www.youtube.com/playlist?list=PLyivlXe3FjG3iJ25wPWmZP8bVi-410s6E": 6},
 {"https://www.youtube.com/playlist?list=PLehuLRPyt1HxuYpdlW4KevYJVOSDG3DEz":21}
]

In [14]:
base_path = "//content//drive//MyDrive//YouTube//Download_video//"

for URL in URLS:
    if isinstance(URL, dict) and len(URL) == 1:
        url = list(URL.keys())[0]
        playlist_start = list(URL.values())[0]
    elif isinstance(URL, str):
        url = URL
        playlist_start = 1
    else:
        print('There are some errors in URLS')
        break

    if not url_list_add(url):
        print(url)
        print('-' * 50, '\n')

        download_video_by_pytube(url, base_path, playlist_start=playlist_start, skip_download=False, max_resolution=1080)

https://www.youtube.com/playlist?list=PLD9Qr3ozd9P0cIvxMOybbeyrVr6ki2cvB already exists
https://www.youtube.com/playlist?list=PLD9Qr3ozd9P0cIvxMOybbeyrVr6ki2cvB
-------------------------------------------------- 

/content/drive/MyDrive/YouTube/Download_video/آموزش-کوبرنتیز_Saeid-Bostandoust
Folder created successfully.
001   کارگاه آموزش کوبرنتیز - قسمت اول
eDbCM6sbcXY is age restricted, and can't be accessed without logging in.
002   کارگاه آموزش کوبرنتیز - قسمت دوم
medium 360p
hd720 720p
720
Downloaded resolution is: 720p
Subtitles are disabled for: 002_کارگاه آموزش کوبرنتیز - قسمت دوم
003   کارگاه آموزش کوبرنتیز - قسمت سوم
medium 360p
hd720 720p
720
Downloaded resolution is: 720p
Subtitles are disabled for: 003_کارگاه آموزش کوبرنتیز - قسمت سوم
https://www.youtube.com/playlist?list=PLyivlXe3FjG3iJ25wPWmZP8bVi-410s6E already exists
https://www.youtube.com/playlist?list=PLyivlXe3FjG3iJ25wPWmZP8bVi-410s6E
-------------------------------------------------- 

/content/drive/MyDrive/YouTu